In [ ]:
import os
from PIL import Image

#Resizing all of our data into 32x32 bmps so we can use that to train our model
def resize_images(input_dir, output_dir, size=(32, 32)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(input_dir):
        class_dir = os.path.join(input_dir, class_name)
        output_class_dir = os.path.join(output_dir, class_name)
        if not os.path.exists(output_class_dir):
            os.makedirs(output_class_dir)

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = Image.open(img_path).convert('L')  # Convert to grayscale
            img_resized = img.resize(size)
            img_resized.save(os.path.join(output_class_dir, img_name))

# Example usage for my specific case:
resize_images('dataset_final', 'dataset_resized', size=(32, 32))

In [ ]:
import tensorflow as tf
import numpy as np
import scipy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset paths, change to yours as needed
train_path = "dataset_resized"
test_path = "dataset_test"

datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load training data
train_datagen = datagen.flow_from_directory(
    train_path,
    target_size=(32, 32),
    batch_size=32,
    color_mode="grayscale",
    class_mode="categorical",
    shuffle=True
)

# Load testing data
test_datagen = datagen.flow_from_directory(
    test_path,
    target_size=(32, 32),
    batch_size=32,
    color_mode="grayscale",
    class_mode="categorical",
    shuffle=False
)

# Convert training dataset to NumPy arrays
train_images, train_labels = [], []

for img_batch, label_batch in train_datagen:
    train_images.append(img_batch)
    train_labels.append(label_batch)

    if len(train_images) * train_datagen.batch_size >= train_datagen.samples:
        break  # Stop after processing all images

# Convert lists to NumPy arrays
train_images = np.concatenate(train_images, axis=0)
train_labels = np.concatenate(train_labels, axis=0)

# Convert lists to single NumPy arrays
train_images = np.array(train_images) 
train_labels = np.array(train_labels)  


# Print dataset shapes to double check that it was done correctly
print(f"Train Images shape: {train_images.shape}")  # (num_samples, 32, 32, 1)
print(f"Train Labels shape: {train_labels.shape}")  # (num_samples, num_classes)


Found 1712 images belonging to 3 classes.
Found 219 images belonging to 3 classes.
Train Images shape: (1712, 32, 32, 1)
Train Labels shape: (1712, 3)


In [9]:
from sklearn.model_selection import train_test_split

# Shuffle dataset
indices = np.arange(train_images.shape[0])
np.random.shuffle(indices)
images, labels = train_images[indices], train_labels[indices]

# Split dataset (70% train, 15% validation, 15% test)
train_imgs, val_imgs, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.15, random_state=42, stratify=train_labels
)

print(f"Train Images shape: {train_imgs.shape}")  # (num_samples, 32, 32, 1)
print(f"Train Labels shape: {train_labels.shape}")  # (num_samples, num_classes)


Train Images shape: (1455, 32, 32, 1)
Train Labels shape: (1455, 3)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), strides=2, activation="relu", input_shape=(32, 32, 1)),

    tf.keras.layers.Conv2D(64, (3, 3), strides=2, activation="relu"),

    tf.keras.layers.Conv2D(128, (3, 3), strides=2, activation="relu"),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation="softmax")  # Output layer
])
model.summary()

# Note: strides of two were used to avoid using max pooling because .tmdl files don't support that

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 15, 15, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 1152)              0         
                                                                 
 dense (Dense)               (None, 64)                73792     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 1

In [ ]:
# Compile model
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Train model -- quick and dirty shortcut was to use the testing data as validation data, but validation set could be used instead
history = model.fit(
    train_datagen,
    validation_data=test_datagen,
    epochs=15
)

# Evaluate on test set
test_loss, test_acc = model.evaluate(test_datagen)
print(f"\nTest Accuracy: {test_acc:.4f}")

# Save model
model.save("rock_paper_scissors_cnn.h5")

Epoch 1/15
54/54 [==============================] - 1s 13ms/step - loss: 1.0901 - accuracy: 0.4071 - val_loss: 1.0524 - val_accuracy: 0.6758
Epoch 2/15
54/54 [==============================] - 1s 10ms/step - loss: 0.9883 - accuracy: 0.5199 - val_loss: 0.6289 - val_accuracy: 0.7215
Epoch 3/15
54/54 [==============================] - 1s 10ms/step - loss: 0.7774 - accuracy: 0.6367 - val_loss: 0.3381 - val_accuracy: 0.9361
Epoch 4/15
54/54 [==============================] - 1s 10ms/step - loss: 0.6478 - accuracy: 0.6951 - val_loss: 0.1821 - val_accuracy: 0.9635
Epoch 5/15
54/54 [==============================] - 1s 10ms/step - loss: 0.5618 - accuracy: 0.7442 - val_loss: 0.1487 - val_accuracy: 0.9269
Epoch 6/15
54/54 [==============================] - 1s 10ms/step - loss: 0.4791 - accuracy: 0.7909 - val_loss: 0.1463 - val_accuracy: 0.9498
Epoch 7/15
54/54 [==============================] - 1s 10ms/step - loss: 0.4352 - accuracy: 0.8096 - val_loss: 0.1433 - val_accuracy: 0.9315
Epoch 8/15
54

c:\Users\Srive\miniconda3\envs\hello\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Used to convert our h5 file into a .tmdl file
!python TinyMaix/tools/h5_to_tflite.py rock_paper_scissors_cnn.h5 model.tflite 0
!python TinyMaix/tools/tflite2tmdl.py model.tflite model.tmdl fp32 1 32,32,1 3 0 